# Chapter 5 SVD: Basic Properties

In [1]:
# numerical and scientific computing libraries
import numpy as np
import scipy as sp

# plotting libraries
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# for pretty printing
np.set_printoptions(4, linewidth=100, suppress=True)

In [5]:
# create a random matrix of size m x n with the rank <= k <= min(m, n).
def create_random_matrix(m: int, n: int, k: int) -> np.ndarray:
    if k > min(m, n):
        raise ValueError("k must be less than or equal to min(n, m)")
    A = np.random.randn(m, k)
    B = np.random.randn(k, n)
    return A@B

Define a function returning a compact SVD

In [35]:
def SVD_compact(A: np.ndarray, eps) -> np.ndarray:
    # perform QR decomposition on the matrix and return the Q matrix and the R matrix.
    Uf, sf, Vfh = np.linalg.svd(A, full_matrices=False)
    Vf = Vfh.T

    # calculate the rank of the matrix
    rank_A = (np.abs(sf) > eps).sum()
    print("Rank of A:", rank_A)

    # reconstruct the matrix using the SVD
    U = Uf[:, :rank_A]
    s = sf[:rank_A]
    V = Vf[:, :rank_A]

    return U, s, V.T

Let us obtain SVD of a randomly generated matrix.

In [37]:
# set the size of the matrix
m = 20
n = 10
k = 7
eps = 1e-10

# create a random matrix of size n x m with the rank k <= n.
A = create_random_matrix(m, n, k)

# perform QR decomposition on the matrix and return the Q matrix and the R matrix.
U, s, Vh = SVD_compact(A, eps)
V = Vh.T

print(U.shape, s.shape, V.shape)
print(s)

np.allclose(A, U @ np.diag(s) @ V.T)

Rank of A: 7
(20, 7) (7,) (10, 7)
[26.9899 20.9595 19.2893 11.8508  9.4604  5.6491  2.6928]


True

### Pseudoinverse

Let us check a defining property of pseudoinverse: $A A^+ A = A$ and $A^+ A A^+ = A^+$.

In [39]:
Ainv = V @ np.diag(1/s) @ U.T

print(np.allclose(A @ Ainv @ A, A), np.allclose(Ainv @ A @ Ainv, Ainv))

True True


### Symmetric Matrices

Let us compute SVD of various symmetric matrices. 

In [49]:
m = 10
k = 7

# create a random matrix of size m x m with its rank k < m.
A = create_random_matrix(m, m, k)
B = A + A.T
C = A @ A.T

U1, s1, V1h = SVD_compact(B, eps)
V1 = V1h.T

print('B')
print(s1)
print(' ')
print(U1)
print(' ')
print(V1)

U2, s2, V2h = SVD_compact(C, eps)
V2 = V2h.T

print('C')
print(s2)
print(' ')
print(U2)
print(' ')
print(V2)


Rank of A: 10
B
[27.7905 14.8514 12.2962  7.2334  5.8916  4.9883  4.4304  1.6992  1.5105  0.6205]
 
[[-0.2645  0.0309  0.4273 -0.127  -0.361  -0.232  -0.1858 -0.341  -0.4886 -0.396 ]
 [-0.3975  0.4047  0.0523 -0.1428  0.6487 -0.0048  0.2552 -0.0753  0.1161 -0.3873]
 [ 0.0141 -0.1506  0.0628 -0.0035 -0.2733  0.4666 -0.1742  0.1584  0.4825 -0.6265]
 [-0.2239 -0.3729  0.3994  0.1481  0.2285 -0.2914 -0.1391  0.6865 -0.0394 -0.0109]
 [ 0.0933  0.1073 -0.2934 -0.0996 -0.2534 -0.7764  0.1003  0.0742  0.382  -0.2351]
 [ 0.1854  0.3189  0.5371 -0.5434 -0.2155  0.0761  0.2841  0.2149  0.1887  0.2557]
 [-0.4822  0.4038 -0.0508  0.0165 -0.1334  0.0099 -0.6261  0.0308  0.2705  0.3428]
 [ 0.0104  0.47   -0.3313  0.0945 -0.2373  0.1474  0.1026  0.5636 -0.4703 -0.1813]
 [ 0.3261 -0.0931 -0.2165 -0.6428  0.3102 -0.0254 -0.52    0.0884 -0.1881 -0.1205]
 [-0.5827 -0.4097 -0.3426 -0.4625 -0.2025  0.1119  0.2949  0.0604 -0.047   0.1224]]
 
[[-0.2645 -0.0309  0.4273 -0.127   0.361  -0.232   0.1858 -0.341   

As you can see from the experiment, the singular vectors of $B$ satisfy $\mathbf{u}_i = \mathbf{v}_i$ or $\mathbf{u}_i = -\mathbf{v}_i$. For these vectors, $B \mathbf{v}_i = \sigma_i \mathbf{u}_i = (\pm \sigma_i) \mathbf{v}_i$ holds which implies that  ($\sigma_i, \mathbf{v}_i$) or ($-\sigma_i, \mathbf{v}_i$) is an eigenpair of $B$. In contrast, $C$ shows $\mathbf{u}_i = \mathbf{v}_i$ for all $i$. Therefore, a pair of a sigular value and its right-singular vector is an eigenpair for $C$. That is, both matrices are symmetric, but $B$ may have negative eigenvalues whereas $C$ always has non-negative eigenvalues. $C$ is positive semi-definite in fact.

In addition, for a square matrix $A$ not of full rank, $AA^\top$ keeps the rank of $A$ as the theory suggested, but $A+A^\top$ has an increased full rank in this random experiment. Generally, we have no concrete result comparing ranks of $A$ and $A+A^\top$.  